In [ ]:
from profiling.funs import load, my_key, _save
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def check(keywords):
    def find(x):
        return any(
            [word in x for word in keywords]
        )
    return find

In [ ]:
def check_just(keywords):
    def find(x):
        return any(
            [word == x for word in keywords]
        )
    return find

In [ ]:
def from_set(fz):
    return {name:value for (name,value) in fz}

# TensorFlow

In [ ]:
def no_idle(df):
    df = df[df['Operation']!='IDLE']
    
    return df

In [ ]:
def store_tf(num):
    dct = load('profiling/results/data_{}.tflow'.format(num))

    def total_on(df, words):
        mask = df['Operation'].apply(check(words))
        return df[mask]['Avg. self-time (us)'].sum()
    
    def get(key):
        fsk=from_set(key)
        name = fsk['model_str']
        
        num=total_on(dct[key], fsk['model_str'].upper())
        
        fsk['total_ms'] = num
       
        return fsk
    
    big_list = filter(None, [get(key) for key in dct])
    df = pd.DataFrame(data = big_list)
    
    df = df[['model_str', 'hp', 'numf', 'batch', 'nodes', 'total_ms']]
    
    _save(df, 'datasets/total_{}.{}'.format(num, frame))

In [ ]:
store_tf(num=1)

In [ ]:
store_tf(num=2)

In [ ]:
store_tf(num=3)

In [ ]:
load('datasets/total_{}.{}'.format(1, 'tflow'))

In [ ]:
load('datasets/total_{}.{}'.format(2, 'tflow'))

# PyTorch

In [ ]:
ops_tor = {
    'conv1d': ['conv1d', 'MkldnnConvolutionBackward'],
    
    'conv2d':['conv2d', 'MkldnnConvolutionBackward'],
    
    'avg1d': ['avg_pool1d'],
    
    'avg2d': ['avg_pool2d'],
    
    'max1d': ['max_pool1d'],
    
    'max2d': ['max_pool2d'],
    
    'dense': ['addmm', 'AddmmBackward'],
    
    'norm1d': ['NativeBatchNormBackward', 'batch_norm', 'native_batch_norm'],
    
    'norm2d': ['NativeBatchNormBackward', 'batch_norm', 'native_batch_norm']
}

In [ ]:
def df_total(dct, ops, clean):
    def get(key):
        key_dct = from_set(key)
        name = key_dct['model_str']
        
        total = clean(dct[key], ops[name])['total_ms'].sum()
        
        key_dct['total_ms'] = total
        
        return key_dct
    
    big_list = [get(key) for key in dct]
    df = pd.DataFrame(data = big_list)
    
    df = df[['model_str', 'hp', 'numf', 'batch', 'nodes', 'total_ms']]
    
    return df
    

In [ ]:
def clean_tor(_df, words):
    df = _df.reset_index()
    df = df[['Name', 'CPU time avg (us)']]
    df['CPU time avg (us)'] = df['CPU time avg (us)']/1000
    df.columns = ['op', 'total_ms']
    df = df.sort_values(by='total_ms', ascending=False).reset_index(drop=True)
    
    mask = df['op'].apply(check_just(words))
    return df[mask]

In [ ]:
def store_tor(num, ops, clean):
    dct = load('profiling/results/data_{}.torch'.format(num))

    df = df_total(dct, ops, clean)

    _save(df, 'datasets/total_{}.torch'.format(num))

In [ ]:
def store_big(num, the_col):
    dct = load('profiling/results/data_{}.torch'.format(num))

    df = big_total(dct, the_col)

    _save(df, 'datasets/big_total_{}.torch'.format(num))

In [ ]:
def big_total(dct, the_col):
    def get(key):
        key_dct = from_set(key)
        name = key_dct['model_str']
        
        total = dct[key][the_col].sum()/1000
        
        key_dct['total_ms'] = total
        
        return key_dct
    
    big_list = filter(None, [get(key) for key in dct])
    df = pd.DataFrame(data = big_list)
    
    df = df[['model_str', 'hp', 'numf', 'batch', 'nodes', 'total_ms']]
    
    return df

In [ ]:
store_tor(num=1, ops=ops_tor, clean=clean_tor)

In [ ]:
store_tor(num=2, ops=ops_tor, clean=clean_tor)

In [ ]:
store_tor(num=3, ops=ops_tor, clean=clean_tor)

In [ ]:
load('datasets/total_{}.{}'.format(1, 'torch'))

In [ ]:
load('datasets/total_{}.{}'.format(2, 'torch'))

In [ ]:
load('datasets/total_{}.{}'.format(3, 'torch'))

In [ ]:
store_big(num=1,frame='torch', the_col='CPU time avg (us)')

In [ ]:
store_big(num=2,frame='torch', the_col='CPU time avg (us)')

In [ ]:
store_big(num=3,frame='torch', the_col='CPU time avg (us)')

In [ ]:
load('datasets/big_total_{}.{}'.format(1, 'torch'))

In [ ]:
load('datasets/big_total_{}.{}'.format(2, 'torch'))

In [ ]:
load('datasets/big_total_{}.{}'.format(3, 'torch'))